In [1]:
import os, sys, requests, json, pathlib, datetime, shapely
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import r5py
import geocube
from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial
os.chdir('C:/Users/tgwon/wri/cif/cities-cif')
sys.path.append('C:/Users/tgwon/wri/cif/cities-cif')
os.environ['GOOGLE_APPLICATION_USER'] = 'script-service-account@wri-gee.iam.gserviceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/Users/tgwon/.google/credkey.json'
sys.path = ['C:\\Users\\tgwon\\wri\\indicators', 'C:\\Users\\tgwon\\wri\\cif\\cities-cif', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\python310.zip', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\DLLs', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib\\site-packages', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib\\site-packages\\win32', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib\\site-packages\\win32\\lib', 'C:\\Users\\tgwon\\anaconda3\\envs\\cities-cif\\lib\\site-packages\\Pythonwin',]

from city_metrix.layers import WorldPop, UrbanExtents
from city_metrix.layers.layer_geometry import GeoExtent

Authenticating to GEE with configured credentials file.


In [2]:
# Get geodataframe of amenity points from file

amenity_points = gpd.GeoDataFrame.from_file('C:/Users/tgwon/wri/indicators/amenitypoints/points__public_open_space__ARG-Buenos_Aires__urbextbound.geojson')
amenity_points

,index,id,amenityclass_road,geometry
0,0,0,True,POINT (-58.38529 -34.60049)
1,1,1,True,POINT (-58.38488 -34.60157)
2,2,2,True,POINT (-58.38455 -34.60046)
3,3,3,True,POINT (-58.38451 -34.60263)
4,4,4,True,POINT (-58.38433 -34.60054)
...,...,...,...,...
23589,31699,31699,True,POINT (-58.4904 -34.57111)
23590,31700,31700,True,POINT (-58.4898 -34.57166)
23591,31704,31704,True,POINT (-57.94129 -34.90402)
23592,31705,31705,True,POINT (-57.94122 -34.90409)


In [3]:
# Get admin bound from API, and get urban extent as geodataframe

CITYDATA_URL = 'https://cities-data-api.wri.org/cities'
citydata = requests.get(CITYDATA_URL).json()
focal_cities = [c for c in citydata['cities'] if c['name']=='Buenos Aires']
city = focal_cities[0]

url = city['layers_url']['geojson']
gpd.GeoDataFrame.from_file(url)
city_admin = gpd.GeoDataFrame.from_file(url)

city_urbext = UrbanExtents().get_data(GeoExtent(city_admin.total_bounds)).to_crs('EPSG:4326').dissolve()

Skipping field bbox: unsupported OGR type: 3
Skipping field bbox: unsupported OGR type: 3


In [4]:
# Create geodataframe of population-pixel points by vectorizing WorldPop raster. Include only those within the boundary of interest.

bbox = GeoExtent(city_urbext.total_bounds)
utm_crs = bbox.as_utm_bbox().crs
worldpop_data = WorldPop(agesex_classes=[]).get_data(bbox)
wp_df = worldpop_data.drop_vars(['time']).to_dataframe().reset_index()
pop_points = gpd.GeoDataFrame(wp_df.population, geometry=gpd.points_from_xy(wp_df.x,wp_df.y))
pop_points_geogr = pop_points.set_crs(utm_crs).to_crs('EPSG:4326')

# Clip to boundary
pop_points_clipped = pop_points_geogr.loc[pop_points_geogr.intersects(city_urbext.geometry[0])]

Extracting layer world pop from Google Earth Engine for bbox :
[########################################] | 100% Completed | 3.85 sms


In [8]:
pop_points_clipped

,population,geometry
15878,3.872722,POINT (-58.09357 -35.04939)
15879,4.176632,POINT (-58.09247 -35.0494)
15880,4.160919,POINT (-58.09137 -35.04941)
15881,3.464365,POINT (-58.09028 -35.04941)
15882,3.208935,POINT (-58.08918 -35.04942)
...,...,...
908804,6.233791,POINT (-58.78115 -34.29107)
908805,5.842262,POINT (-58.78006 -34.29109)
909872,6.098891,POINT (-58.78765 -34.29008)
909873,6.265943,POINT (-58.78656 -34.29009)


In [5]:
b = r5py.TransportNetwork(pathlib.Path('C:/Users/tgwon/wri/indicators/pbf/ARG-Buenos_Aires__urbextbound.osm.pbf'))

In [185]:
# This way takes 402 seconds to do 100 rows
'''%%time
res = pd.DataFrame({'to_id': pop_points_clipped.index, 'numpixels': [0] * len(pop_points_clipped)})
for i in list(range(len(amenity_points)))[:100]:
    if i % 1 == 0:
        print(i, end=' ', flush=True)
    d = r5py.TravelTimeMatrix(transport_network=b, origins=gpd.GeoDataFrame({'id': [i], 'geometry': amenity_points.iloc[[i]].centroid}), destinations=gpd.GeoDataFrame({'id': range(len(pop_points_clipped)), 'geometry': pop_points_clipped.geometry}), transport_modes=[r5py.TransportMode.WALK], max_time=datetime.timedelta(minutes=15), max_time_walking=datetime.timedelta(minutes=15))
    res.numpixels = res.numpixels + ((d.travel_time / d.travel_time).fillna(0))'''

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 CPU times: total: 18min 12s
Wall time: 6min 42s


In [193]:
# This way takes 90 seconds to do 100 rows
'''%%time
res = pd.DataFrame({'to_id': pop_points_clipped.index, 'numpixels': [0] * len(pop_points_clipped)})

d = r5py.TravelTimeMatrix(transport_network=b, origins=gpd.GeoDataFrame({'id': range(100), 'geometry': amenity_points.iloc[:100].centroid}), destinations=gpd.GeoDataFrame({'id': range(len(pop_points_clipped)), 'geometry': pop_points_clipped.geometry}), transport_modes=[r5py.TransportMode.WALK], max_time=datetime.timedelta(minutes=15), max_time_walking=datetime.timedelta(minutes=15))
d['num_accessible'] = (d.travel_time / d.travel_time).fillna(0)
num_accessible = d.pivot(index='to_id', columns='from_id', values='num_accessible').sum(axis=1, skipna=True)
res.numpixels = res.numpixels + num_accessible'''

CPU times: total: 1min 38s
Wall time: 1min 30s


In [ ]:
%%time

def num_accessible_onechunk(i, j):
    d = r5py.TravelTimeMatrix(transport_network=b, origins=gpd.GeoDataFrame({'id': range(i, j), 'geometry': amenity_points.iloc[i:j].centroid}), destinations=gpd.GeoDataFrame({'id': range(len(pop_points_clipped)), 'geometry': pop_points_clipped.geometry}), transport_modes=[r5py.TransportMode.WALK], max_time=datetime.timedelta(minutes=15), max_time_walking=datetime.timedelta(minutes=15))
    d['num_accessible'] = (d.travel_time / d.travel_time).fillna(0)
    num_accessible = d.pivot(index='to_id', columns='from_id', values='num_accessible').sum(axis=1, skipna=True)
    return num_accessible

CHUNKSIZE = 100
res = pd.DataFrame({'to_id': pop_points_clipped.index, 'num_accessible': [0] * len(pop_points_clipped)})
for chunk in range((len(amenity_points) // CHUNKSIZE) + 1):
    print(chunk * CHUNKSIZE, end=' ', flush=True)
    i, j = chunk * CHUNKSIZE, min((chunk + 1) * CHUNKSIZE, len(amenity_points))
    num_accessible = num_accessible_onechunk(i, j)
    res.num_accessible = res.num_accessible + num_accessible

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 17800 17900 18000 18100 18200 18300 18400 18

In [144]:
%%time
# Take result dataframe and merge results back into the population-pixl geodataframe

res = res.set_index('to_id')
pop_points['accessible_points'] = 0
pop_points.loc[pop_points_clipped.index, 'accessible_points'] = res['num_accessible']
pop_points.accessible_points = pop_points.accessible_points.fillna(0).astype(int)

CPU times: total: 46.9 ms
Wall time: 43.1 ms


In [165]:
# Convert pop-pixel gdf with number-of-accessible-points data to raster

geo_grid = make_geocube(
    vector_data=pop_points.set_crs(utm_crs),
    measurements=['accessible_points'],
    like = worldpop_data,
    rasterize_function=rasterize_points_griddata,
)
geo_grid.rio.to_raster('C:/Users/tgwon/wri/indicators/ba_test.tif', engine='GeoTIFF')